In [28]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [37]:
def successor(X, s, r, delta, m, T):
    h = T/m
    z = np.random.normal()
    return X * np.exp((r - delta - (0.5 * s**2)) * h + s * np.sqrt(h) * z)

def black_scholes(X0, s, r, delta, m, T):
    X = [X0]
    for i in range(m):
        X.append(successor(X[-1], s, r, delta, m, T))
    return X

def put_func(a, K, r, t):
    """
    Payoff function for a Bermudan put option
    """
    return np.exp(-r*t) * np.maximum(K - a, 0)

def call_func(a, K, r, t):
    """
    Payoff function for a Bermudan call option
    """
    return np.exp(-r*t) * np.maximum(a - K, 0)

def high_estimator(payoff_func, current, nextt, K, r, t):
    return max(payoff_func(current, K, r, t), np.mean(nextt))

def low_estimator(payoff_func, current, nextt, K, r, t):
    b = len(nextt)
    v_ = np.zeros(b)
    for k in range(b):
        new = np.concatenate([nextt[:k], nextt[k+1:]]) # remove the k-th element
        if np.mean(new) <= payoff_func(current, K, r, t):
            v_[k] = payoff_func(current, K, r, t)
        else:
            v_[k] = nextt[k]
    return np.mean(v_)


In [38]:
def rtree(X0, m, b, s, r, delta, K, T, payoff_func):
    h = T/m

    w = np.zeros(m+1).astype(int)     # to know at which branch we are
    # Values of the estimators
    v = np.zeros((b, m+1))   
    V = np.zeros((b, m+1))   

    # Root nodes
    v[0, 0] = X0    
    V[0, 0] = X0

    # Initialize the first branches
    for j in range(1, m+1):
        v[0, j] = successor(v[0, j-1], s, r, delta, m, T)
        V[0, j] = successor(V[0, j-1], s, r, delta, m, T)
    
    j = m
    while j >= 1:
        if j == m and w[j] < b-1:
            v[w[j], j] = payoff_func(v[w[j], j], K, r, j*h)
            V[w[j], j] = payoff_func(V[w[j], j], K, r, j*h)
            v[w[j]+1, j] = successor(v[w[j-1], j-1], s, r, delta, m, T)
            V[w[j]+1, j] = successor(V[w[j-1], j-1], s, r, delta, m, T)
            w[j] += 1
        elif j == m and w[j] == b-1:
            v[w[j], j] = payoff_func(v[w[j], j], K, r, j*h)
            V[w[j], j] = payoff_func(V[w[j], j], K, r, j*h)
            w[j] = 0
            j -= 1
        elif j < m and w[j] < b-1:
            V[w[j], j] = high_estimator(payoff_func, V[w[j], j], V[:, j+1], K, r, j*h)
            v[w[j], j] = low_estimator(payoff_func, v[w[j], j], v[:, j+1], K, r, j*h)
            if j > 0:
                w[j] += 1
                v[w[j], j] = successor(v[w[j-1], j-1], s, r, delta, m, T)
                V[w[j], j] = successor(V[w[j-1], j-1], s, r, delta, m, T)
                for i in range(j+1, m+1):
                    v[0, i] = successor(v[w[j], i-1], s, r, delta, m, T)
                    V[0, i] = successor(V[w[j], i-1], s, r, delta, m, T)
                j = m
            else:
                break
        elif j < m and w[j] == b-1:
            v[w[j], j] = low_estimator(payoff_func, v[w[j], j], v[:, j+1], K, r, j*h)
            V[w[j], j] = high_estimator(payoff_func, V[w[j], j], V[:, j+1], K, r, j*h)
            w[j] = 0
            j -= 1
    return low_estimator(payoff_func, v[0, 0], v[:, 1], K, r, 0), high_estimator(payoff_func, V[0, 0], V[:, 1], K, r, 0)

In [39]:
def monte_carlo(n, X0, m, b, s, r, delta, K, T, payoff_func):
    eff_low = []
    eff_high = []
    for _ in tqdm(range(n)):
        eff_low.append(rtree(X0, m, b, s, r, delta, K, T, payoff_func)[0])
        eff_high.append(rtree(X0, m, b, s, r, delta, K, T, payoff_func)[1])
    return eff_low, eff_high

In [42]:
n = 100
X0 = 100
m = 3
b = 50
s = 0.2
r = 0.1
delta = 0.05
K = 100
T = 1
eff_low, eff_high = monte_carlo(n, X0, m, b, s, r, delta, K, T, put_func)

100%|██████████| 100/100 [35:12<00:00, 21.12s/it]


In [43]:
parameters = [n, X0, m, b, s, r, delta, K, T]
np.save('parametersb50.npy', parameters)
np.save('runb50.npy', [eff_low, eff_high])

In [58]:
mean_v = np.mean(eff_low)
mean_V = np.mean(eff_high)

z = 1.96

std_v = 1/(n-1) * np.sum((eff_low - mean_v)**2)
std_V = 1/(n-1) * np.sum((eff_high - mean_V)**2)
print('90% confidence interval: [', mean_v - z * std_v / np.sqrt(n), mean_V + z * std_V / np.sqrt(n),']')

print('Low estimator:', mean_v)
print('High estimator:', mean_V)
print('Std error for the low estimator:', z * std_v/np.sqrt(n))
print('Std for the high estimator:', z * std_V/np.sqrt(n))
print('Point estimate:', 0.5 * (mean_v + mean_V))

90% confidence interval: [ 5.62260720089785 5.962591632886328 ]
Low estimator: 5.731102600680967
High estimator: 5.862991122675001
Std error for the low estimator: 0.10849539978311688
Std for the high estimator: 0.09960051021132751
Point estimate: 5.797046861677984


With variance reduction techniques